<a href="https://colab.research.google.com/github/Shyam657/Youtube-Tutorials/blob/main/Langchain_Mine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import IFrame

IFrame('https://www.youtube.com/embed/38aMTXY2usU', width=560, height=315)

In [ ]:
!pip install -q --upgrade langchain langchain-openai langchain-core langchain_community docx2txt pypdf  langchain_chroma sentence_transformers langchain-together docx2txt pypdf unstructured together

In [ ]:
#together ai api key 024254d1fcd8eee3a12258e40260e8411c8afcec22c65baca46f40aed1c60904

In [ ]:
import os
os.environ["TOGETHERAI_API_KEY"] = "024254d1fcd8eee3a12258e40260e8411c8afcec22c65baca46f40aed1c60904"

In [ ]:
import getpass
import os

if not os.environ.get("TOGETHER_API_KEY"):
  os.environ["TOGETHER_API_KEY"] = getpass.getpass("Enter API key for Together AI: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo", model_provider="together")

Enter API key for Together AI: ··········


In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_PROJECT"] = "langchain-course"

In [ ]:
from langchain_together import ChatTogether

In [ ]:
llm_response = model.invoke("Tell me a joke")

llm_response

AIMessage(content='A man walked into a library and asked the librarian, "Do you have any books on Pavlov\'s dogs and Schrödinger\'s cat?" \n\nThe librarian replied, "It rings a bell, but I\'m not sure if it\'s here or not."', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 39, 'total_tokens': 94, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-63f6d25c-66f7-4749-a78d-29338e161f68-0', usage_metadata={'input_tokens': 39, 'output_tokens': 55, 'total_tokens': 94, 'input_token_details': {}, 'output_token_details': {}})

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(llm_response)

'A man walked into a library and asked the librarian, "Do you have any books on Pavlov\'s dogs and Schrödinger\'s cat?" \n\nThe librarian replied, "It rings a bell, but I\'m not sure if it\'s here or not."'

In [ ]:
chain = model | output_parser
chain.invoke("Tell me a joke")

'A man walked into a library and asked the librarian, "Do you have any books on Pavlov\'s dogs and Schrödinger\'s cat?" \n\nThe librarian replied, "It rings a bell, but I\'m not sure if it\'s here or not."'

In [ ]:
from typing import List, Optional
from pydantic import BaseModel, Field

class MobileReview(BaseModel):
    phone_model: str = Field(description="Name and model of the phone")
    rating: Optional[float] = Field(description="Overall rating out of 5", default=None)  # Make rating optional with a default value
    pros: List[str] = Field(description="List of positive aspects", default=[]) # Make pros optional with a default value
    cons: List[str] = Field(description="List of negative aspects", default=[]) # Make cons optional with a default value
    summary: Optional[str] = Field(description="Brief summary of the review", default=None)  # Make summary optional with a default value

review_text = """
Just got my hands on the new Galaxy S21 and wow, this thing is slick! The screen is gorgeous,
colors pop like crazy. Camera's insane too, especially at night - my Insta game's never been
stronger. Battery life's solid, lasts me all day no problem.

Not gonna lie though, it's pretty pricey. And what's with ditching the charger? C'mon Samsung.
Also, still getting used to the new button layout, keep hitting Bixby by mistake.

Overall, I'd say it's a solid 4 out of 5. Great phone, but a few annoying quirks keep it from
being perfect. If you're due for an upgrade, definitely worth checking out!
"""

structured_llm = model.with_structured_output(MobileReview)
output = structured_llm.invoke(review_text)

# Check if the output is not None before accessing attributes
if output:
    print(output.summary)
else:
    print("The model was unable to extract information from the review text.")

Great phone, but a few annoying quirks keep it from being perfect.


In [ ]:
output.pros

['screen is gorgeous', 'colors pop', "camera's insane", "battery life's solid"]

In [ ]:
output.rating

4.0

In [ ]:
output.phone_model

'Galaxy S21'

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
prompt.invoke({"topic": "programming"})

ChatPromptValue(messages=[HumanMessage(content='Tell me a short joke about programming', additional_kwargs={}, response_metadata={})])

In [ ]:
chain = prompt | model | output_parser
chain.invoke({"topic": "programer"})

'Why do programmers prefer dark mode?\n\nBecause light attracts bugs.'

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_together import ChatTogether
# Define the prompt
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")

In [ ]:
# Querying chat models with Together AI

from langchain_together import ChatTogether

# choose from our 50+ models here: https://docs.together.ai/docs/inference-models
chat = ChatTogether(
    # together_api_key="YOUR_API_KEY",
    model="meta-llama/Llama-3-70b-chat-hf",
)


# Compose the chain
chain = prompt | chat | output_parser

# Use the chain
result = chain.invoke({"topic": "programming"})
print(result)



# stream the response back from the model
#for m in chat.stream("Tell me fun things to do in NYC"):
 #   print(m.content, end="", flush=True)

# if you don't want to do streaming, you can use the invoke method
# chat.invoke("Tell me fun things to do in NYC")


Why do programmers prefer dark mode?

Because light attracts bugs.


In [ ]:
# Compose the chain
chain = prompt | chat | output_parser

# Use the chain
result = chain.invoke({"topic": "programming"})
print(result)

Why do programmers prefer dark mode?

Because light attracts bugs.


In [ ]:
template = ChatPromptTemplate([
    ("system", "You are a helpful assistant that tells jokes."),
    ("human", "Tell me about {topic}")
])

prompt_value = template.invoke(
    {
        "topic": "programming"
    }
)
prompt_value

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant that tells jokes.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me about programming', additional_kwargs={}, response_metadata={})])

In [ ]:
template = ChatPromptTemplate([
    ("system", "You are a helpful assistant that tells jokes."),
    ("human", "Tell me about {topic}")
])

prompt_value = template.invoke(
    {
        "topic": "programming"
    }
)
prompt_value

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant that tells jokes.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me about programming', additional_kwargs={}, response_metadata={})])

In [ ]:
model.invoke(prompt_value)

AIMessage(content="Programming, it's a world of code, bugs, and endless coffee cups. But don't worry, I'm here to help you navigate it.\n\nProgramming is the process of designing, writing, testing, and maintaining the source code of computer programs. It's like writing a recipe for your computer to follow, but instead of ingredients, you use programming languages like Python, Java, or C++.\n\nImagine you're trying to make a peanut butter and jelly sandwich. You need to tell the computer exactly what to do, step by step:\n\n1. Get two slices of bread.\n2. Spread peanut butter on one slice.\n3. Spread jelly on the other slice.\n4. Put the two slices together.\n\nIn programming, you would write code that says something like:\n\n`bread = get_bread()`\n`peanut_butter = spread_peanut_butter(bread)`\n`jelly = spread_jelly(bread)`\n`sandwich = combine_slices(peanut_butter, jelly)`\n\nBut, just like in real life, things don't always go as planned. You might get a bug in your code, like a typo o

In [ ]:
!mkdir docs

mkdir: cannot create directory ‘docs’: File exists


In [ ]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from typing import List
from langchain_core.documents import Document

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

docx_loader = Docx2txtLoader("/content/docs/GreenGrow Innovations_ Company History.docx")
documents = docx_loader.load()

print(len(documents))

splits = text_splitter.split_documents(documents)

print(f"Split the documents into {len(splits)} chunks.")

1
Split the documents into 2 chunks.


In [ ]:
documents[0]

Document(metadata={'source': '/content/docs/GreenGrow Innovations_ Company History.docx'}, page_content="GreenGrow Innovations was founded in 2010 by Sarah Chen and Michael Rodriguez, two agricultural engineers with a passion for sustainable farming. The company started in a small garage in Portland, Oregon, with a simple mission: to make farming more environmentally friendly and efficient.\n\n\n\nIn its early days, GreenGrow focused on developing smart irrigation systems that could significantly reduce water usage in agriculture. Their first product, the WaterWise Sensor, was launched in 2012 and quickly gained popularity among local farmers. This success allowed the company to expand its research and development efforts.\n\n\n\nBy 2015, GreenGrow had outgrown its garage origins and moved into a proper office and research facility in the outskirts of Portland. This move coincided with the development of their second major product, the SoilHealth Monitor, which used advanced sensors to

In [ ]:
splits[1]

Document(metadata={'source': '/content/docs/GreenGrow Innovations_ Company History.docx'}, page_content="The company's breakthrough came in 2018 with the introduction of the EcoHarvest System, an integrated solution that combined smart irrigation, soil monitoring, and automated harvesting techniques. This system caught the attention of large-scale farmers across the United States, propelling GreenGrow to national prominence.\n\n\n\nToday, GreenGrow Innovations employs over 200 people and has expanded its operations to include offices in California and Iowa. The company continues to focus on developing sustainable agricultural technologies, with ongoing projects in vertical farming, drought-resistant crop development, and AI-powered farm management systems.\n\n\n\nDespite its growth, GreenGrow remains committed to its original mission of promoting sustainable farming practices. The company regularly partners with universities and research institutions to advance the field of agricultura

In [ ]:
splits[0].metadata

{'source': '/content/docs/GreenGrow Innovations_ Company History.docx'}

In [ ]:
splits[0].page_content

'GreenGrow Innovations was founded in 2010 by Sarah Chen and Michael Rodriguez, two agricultural engineers with a passion for sustainable farming. The company started in a small garage in Portland, Oregon, with a simple mission: to make farming more environmentally friendly and efficient.\n\n\n\nIn its early days, GreenGrow focused on developing smart irrigation systems that could significantly reduce water usage in agriculture. Their first product, the WaterWise Sensor, was launched in 2012 and quickly gained popularity among local farmers. This success allowed the company to expand its research and development efforts.\n\n\n\nBy 2015, GreenGrow had outgrown its garage origins and moved into a proper office and research facility in the outskirts of Portland. This move coincided with the development of their second major product, the SoilHealth Monitor, which used advanced sensors to analyze soil composition and provide real-time recommendations for optimal crop growth.'

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# 1. Function to load documents from a folder

def load_documents(folder_path: str) -> List[Document]:
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        documents.extend(loader.load())
    return documents

# Load documents from a folder
folder_path = "/content/docs"
documents = load_documents(folder_path)

print(f"Loaded {len(documents)} documents from the folder.")
splits = text_splitter.split_documents(documents)
print(f"Split the documents into {len(splits)} chunks.")

Loaded 5 documents from the folder.
Split the documents into 8 chunks.


In [ ]:
type(splits)

list

In [ ]:
import os
from together import Together

client = Together()

# Extract text content from Document objects and create a list of strings
input_texts = [doc.page_content for doc in splits]

# Pass the list of strings to the embeddings.create method
response = client.embeddings.create(
    model="togethercomputer/m2-bert-80M-8k-retrieval",
    input=input_texts
)


In [ ]:
response.data

In [ ]:
print(f"Created embeddings for {len(response.data)} document chunks.")

Created embeddings for 8 document chunks.


In [ ]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
document_embeddings = embedding_function.embed_documents([split.page_content for split in splits])
document_embeddings[0]

In [ ]:
!pip install langchain_chroma -q

In [ ]:
from langchain_chroma import Chroma
from together import Together

In [ ]:
embedding_function = Together()
collection_name = "my_collection"
vectorstore = Chroma.from_documents(collection_name=collection_name, documents=splits, embedding=embedding_function, persist_directory="./chroma_db")
#db.persist()

print("Vector store created and persisted to './chroma_db'")

AttributeError: 'Together' object has no attribute 'embed_documents'

In [ ]:
%pip install -qU langchain-together

In [ ]:
from langchain_together import TogetherEmbeddings

embeddings = TogetherEmbeddings(
    model="togethercomputer/m2-bert-80M-8k-retrieval",
)

In [ ]:
from langchain_chroma import Chroma
from langchain_together import TogetherEmbeddings
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
'''
# Step 1: Create or load your documents
raw_text = "This is a sample document. It contains multiple sentences. We want to split it into smaller chunks."
doc = Document(page_content=raw_text, metadata={"source": "example"})

# Step 2: Split the documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
splits = text_splitter.split_documents([doc])
'''
# Step 3: Create the embedding function using Together API
embedding_function = TogetherEmbeddings(
    model="togethercomputer/m2-bert-80M-8k-retrieval",
    # api_key="your-together-api-key"  # Uncomment and add your API key if not set as an environment variable
)

# Step 4: Create the vector store
collection_name = "my_collection"
vectorstore = Chroma.from_documents(
    collection_name=collection_name,
    documents=splits,
    embedding=embedding_function,
    persist_directory="./chroma_db"
)

print("Vector store created and persisted to './chroma_db'")

Vector store created and persisted to './chroma_db'


In [ ]:
# 5. Perform similarity search

query = "When was GreenGrow Innovations founded?"
search_results = vectorstore.similarity_search(query, k=2)

print(f"\nTop 2 most relevant chunks for the query: '{query}'\n")
for i, result in enumerate(search_results, 1):
    print(f"Result {i}:")
    print(f"Source: {result.metadata.get('source', 'Unknown')}")
    print(f"Content: {result.page_content}")
    print()


Top 2 most relevant chunks for the query: 'When was GreenGrow Innovations founded?'

Result 1:
Source: /content/docs/GreenGrow Innovations_ Company History.docx
Content: By 2015, GreenGrow had outgrown its garage origins and moved into a proper office and research

Result 2:
Source: /content/docs/GreenGrow Innovations_ Company History.docx
Content: By 2015, GreenGrow had outgrown its garage origins and moved into a proper office and research



In [ ]:

retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
retriever.invoke("When was GreenGrow Innovations founded?")

[Document(id='ecc87323-d0d1-42f3-883e-8d00fff41686', metadata={'source': '/content/docs/GreenGrow Innovations_ Company History.docx'}, page_content='By 2015, GreenGrow had outgrown its garage origins and moved into a proper office and research'),
 Document(id='414d69e4-d43c-43c9-bd34-681d43e60604', metadata={'source': '/content/docs/GreenGrow Innovations_ Company History.docx'}, page_content='By 2015, GreenGrow had outgrown its garage origins and moved into a proper office and research')]

In [ ]:

from langchain_core.prompts import ChatPromptTemplate
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer: """
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} | prompt
)
rag_chain.invoke("When was GreenGrow Innovations founded?")

ChatPromptValue(messages=[HumanMessage(content="Answer the question based only on the following context:\n[Document(id='ecc87323-d0d1-42f3-883e-8d00fff41686', metadata={'source': '/content/docs/GreenGrow Innovations_ Company History.docx'}, page_content='By 2015, GreenGrow had outgrown its garage origins and moved into a proper office and research'), Document(id='414d69e4-d43c-43c9-bd34-681d43e60604', metadata={'source': '/content/docs/GreenGrow Innovations_ Company History.docx'}, page_content='By 2015, GreenGrow had outgrown its garage origins and moved into a proper office and research')]\n\nQuestion: When was GreenGrow Innovations founded?\n\nAnswer: ", additional_kwargs={}, response_metadata={})])

In [ ]:

def docs2str(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:

rag_chain = (
    {"context": retriever | docs2str, "question": RunnablePassthrough()} | prompt
)
rag_chain.invoke("When was GreenGrow Innovations founded?")

ChatPromptValue(messages=[HumanMessage(content='Answer the question based only on the following context:\nBy 2015, GreenGrow had outgrown its garage origins and moved into a proper office and research\n\nBy 2015, GreenGrow had outgrown its garage origins and moved into a proper office and research\n\nQuestion: When was GreenGrow Innovations founded?\n\nAnswer: ', additional_kwargs={}, response_metadata={})])

In [ ]:
rag_chain = (
    {"context": retriever | docs2str, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)
question = "When was GreenGrow Innovations founded?"
response = rag_chain.invoke(question)
print(response)

The provided context does not mention the founding year of GreenGrow. It only mentions that by 2015, GreenGrow had outgrown its garage origins, implying that it was founded before 2015, but the exact year is not specified.


In [ ]:
from langchain_core.messages import HumanMessage, AIMessage
chat_history = []
chat_history.extend([
    HumanMessage(content=question),
    AIMessage(content=response)
])

In [ ]:

chat_history

[HumanMessage(content='When was GreenGrow Innovations founded?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The provided context does not mention the founding year of GreenGrow. It only mentions that by 2015, GreenGrow had outgrown its garage origins, implying that it was founded before 2015, but the exact year is not specified.', additional_kwargs={}, response_metadata={})]

In [ ]:
from langchain_core.prompts import MessagesPlaceholder
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# history_aware_retriever = create_history_aware_retriever(
#     llm, retriever, contextualize_q_prompt
# )
contextualize_chain = contextualize_q_prompt | model | StrOutputParser()
contextualize_chain.invoke({"input": "Where it is headquartered?", "chat_history": chat_history})

'No information is provided about the headquarters of GreenGrow Innovations.'

In [ ]:
from langchain.chains import create_history_aware_retriever
history_aware_retriever = create_history_aware_retriever(
    model, retriever, contextualize_q_prompt
)
history_aware_retriever.invoke({"input": "Where it is headquartered?", "chat_history": chat_history})

[Document(id='3ad5f438-2caf-4699-a5e8-c4d080e8ced3', metadata={'source': '/content/docs/GreenGrow Innovations_ Company History.docx'}, page_content='Today, GreenGrow Innovations employs over 200 people and has expanded its operations to include'),
 Document(id='92d223ca-b292-431b-84f0-65d4566dad64', metadata={'source': '/content/docs/GreenGrow Innovations_ Company History.docx'}, page_content='Today, GreenGrow Innovations employs over 200 people and has expanded its operations to include')]

In [ ]:

retriever.invoke("Where it is headquartered?")

[Document(id='414d69e4-d43c-43c9-bd34-681d43e60604', metadata={'source': '/content/docs/GreenGrow Innovations_ Company History.docx'}, page_content='By 2015, GreenGrow had outgrown its garage origins and moved into a proper office and research'),
 Document(id='ecc87323-d0d1-42f3-883e-8d00fff41686', metadata={'source': '/content/docs/GreenGrow Innovations_ Company History.docx'}, page_content='By 2015, GreenGrow had outgrown its garage origins and moved into a proper office and research')]

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the following context to answer the user's question."),
    #  ("system", "Tell me joke on Programming"),
    ("system", "Context: {context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

question_answer_chain = create_stuff_documents_chain(model, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [ ]:
rag_chain.invoke({"input": "Where it is headquartered?", "chat_history":chat_history})

{'input': 'Where it is headquartered?',
 'chat_history': [HumanMessage(content='When was GreenGrow Innovations founded?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The provided context does not mention the founding year of GreenGrow. It only mentions that by 2015, GreenGrow had outgrown its garage origins, implying that it was founded before 2015, but the exact year is not specified.', additional_kwargs={}, response_metadata={})],
 'context': [Document(id='3ad5f438-2caf-4699-a5e8-c4d080e8ced3', metadata={'source': '/content/docs/GreenGrow Innovations_ Company History.docx'}, page_content='Today, GreenGrow Innovations employs over 200 people and has expanded its operations to include'),
  Document(id='92d223ca-b292-431b-84f0-65d4566dad64', metadata={'source': '/content/docs/GreenGrow Innovations_ Company History.docx'}, page_content='Today, GreenGrow Innovations employs over 200 people and has expanded its operations to include')],
 'answer': "The provided contex

In [ ]:
import sqlite3
from datetime import datetime

DB_NAME = "rag_app.db"

def get_db_connection():
    conn = sqlite3.connect(DB_NAME)
    conn.row_factory = sqlite3.Row
    return conn

def create_application_logs():
    conn = get_db_connection()
    conn.execute('''CREATE TABLE IF NOT EXISTS application_logs
                    (id INTEGER PRIMARY KEY AUTOINCREMENT,
                     session_id TEXT,
                     user_query TEXT,
                     gpt_response TEXT,
                     model TEXT,
                     created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP)''')
    conn.close()

def insert_application_logs(session_id, user_query, gpt_response, model):
    conn = get_db_connection()
    conn.execute('INSERT INTO application_logs (session_id, user_query, gpt_response, model) VALUES (?, ?, ?, ?)',
                 (session_id, user_query, gpt_response, model))
    conn.commit()
    conn.close()

def get_chat_history(session_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('SELECT user_query, gpt_response FROM application_logs WHERE session_id = ? ORDER BY created_at', (session_id,))
    messages = []
    for row in cursor.fetchall():
        messages.extend([
            {"role": "human", "content": row['user_query']},
            {"role": "ai", "content": row['gpt_response']}
        ])
    conn.close()
    return messages

# Initialize the database
create_application_logs()

In [ ]:
import uuid
session_id = str(uuid.uuid4())
chat_history = get_chat_history(session_id)
print(chat_history)
question1 = "When was GreenGrow Innovations founded?"
answer1 = rag_chain.invoke({"input": question1, "chat_history":chat_history})['answer']
insert_application_logs(session_id, question1, answer1, "gpt-4-o-mini")
print(f"Human: {question1}")
print(f"AI: {answer1}\n")

[]
Human: When was GreenGrow Innovations founded?
AI: Unfortunately, the provided context does not mention the exact founding date of GreenGrow. However, it does mention that by 2015, GreenGrow had outgrown its garage origins, suggesting that it was founded sometime before 2015.



In [ ]:

question2 = "Where it is headquartered?"
chat_history = get_chat_history(session_id)
print(chat_history)
answer2 = rag_chain.invoke({"input": question2, "chat_history":chat_history})['answer']
insert_application_logs(session_id, question2, answer2, "gpt-3.5-turbo")
print(f"Human: {question2}")
print(f"AI: {answer2}\n")

[{'role': 'human', 'content': 'When was GreenGrow Innovations founded?'}, {'role': 'ai', 'content': 'Unfortunately, the provided context does not mention the exact founding date of GreenGrow. However, it does mention that by 2015, GreenGrow had outgrown its garage origins, suggesting that it was founded sometime before 2015.'}]
Human: Where it is headquartered?
AI: Unfortunately, the provided context does not mention the location of GreenGrow's headquarters.



In [ ]:

session_id = str(uuid.uuid4())
question = "What is GreenGrow"
chat_history = get_chat_history(session_id)
print(chat_history)
answer = rag_chain.invoke({"input": question, "chat_history":chat_history})['answer']
insert_application_logs(session_id, question, answer, "gpt-3.5-turbo")
print(f"Human: {question}")
print(f"AI: {answer}\n")

[]
Human: What is GreenGrow
AI: I don't have any information about "GreenGrow" in the context you provided. Could you please provide more context or details about what GreenGrow is or what it relates to? This will help me better understand your question and provide a more accurate answer.

